#### iLQR for kinematic example

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
import time
from ocp import *
from costs import *
from ocp_utils import *

import pybullet as p
import pybullet_data

%load_ext autoreload
%autoreload 2
np.set_printoptions(precision=4, suppress=True)


Bad key "text.kerning_factor" on line 4 in
/home/teguh/miniconda3/envs/tf3/lib/python3.8/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


#### Setup pybullet with the urdf

In [2]:
# configure pybullet and load plane.urdf and quadcopter.urdf
#physicsClient = p.connect(p.DIRECT)  # pybullet only for computations no visualisation, faster
physicsClient = p.connect(p.GUI)  # pybullet with visualisation
p.configureDebugVisualizer(p.COV_ENABLE_GUI,0)

p.setAdditionalSearchPath(pybullet_data.getDataPath())

In [3]:
p.resetSimulation()

robot_urdf = "../data/urdf/frankaemika_new/panda_arm.urdf"
robot_id = p.loadURDF(robot_urdf, useFixedBase=1)
joint_limits = get_joint_limits(robot_id, 7)

p.loadURDF('plane.urdf')

#Define the end-effector
link_id = 10
link_name = 'panda_grasptarget_hand'

#Create a ball to show the target
_,_,ballId = create_primitives(radius=0.05)

### Construct the robot system

In [4]:
dt = 0.05
T = 100
dof = 7
sys = URDFRobot(dof=dof, robot_id=robot_id, dt = dt)

#### Set the initial state

In [5]:
q0 = np.random.rand(7)
q0 = np.array([0.,0.,0.,-1.5,0.,1.5,0.])
#q0 = np.array([0.4201, 0.4719, 0.9226, 0.8089, 0.3113, 0.7598, 0.364 ])
x0 = np.concatenate([q0, np.zeros(7)])
sys.set_init_state(x0)

#### Try forward kinematics

In [6]:
pos0, quat0 = sys.compute_ee(x0, link_id)

#Put the ball at the end-effector
p.resetBasePositionAndOrientation(ballId, pos0, quat0)
print(pos0)

[ 0.5477 -0.      0.5465]


#### Set initial control output

In [7]:
#set initial control output to be all zeros
us = np.zeros((T+1,sys.Du))
_ = sys.compute_matrices(x0, us[0])
xs = sys.rollout(us[:-1])

#### Plot initial trajectory

#### Set the regularization cost coefficients Q and R 

In [8]:
Q = np.eye(sys.Dx)*.1
Q[0:sys.dof,0:sys.dof] *= 0.01  #only put cost regularization on the velocity, not on the joint angles
Qf = np.eye(sys.Dx)*1
Qf[0:sys.dof,0:sys.dof] *= 0.01 #only put cost regularization on the velocity, not on the joint angles
R = np.eye(sys.Du)*.1
mu = 1e-6          #regularization coefficient

#### Set end effector reference 

In [51]:
#W and WT: cost coefficients for the end-effector reaching task
p_ref = np.array([0.5, -.4, 0.4])
W = np.eye(3)*1
WT = np.eye(3)*100

In [52]:
p.resetBasePositionAndOrientation(ballId, p_ref, (0,0,0,1))

### iLQR using cost model

#### Define the cost

In [53]:
#The costs consist of: a) state regularization (Q), b) control regularization (R), and c) End-effector reaching task (W)
#Running cost is for the time 0 <= t < T, while terminal cost is for the time t = T

runningStateCost = CostModelQuadratic(sys, Q)
runningControlCost = CostModelQuadratic(sys, None, R)
runningEECost = CostModelQuadraticTranslation(sys,W, link_id,p_ref)
runningCost = CostModelSum(sys, [runningStateCost, runningControlCost, runningEECost])

terminalStateCost = CostModelQuadratic(sys,Qf)
terminalControlCost = CostModelQuadratic(sys, None,R)
terminalEECost = CostModelQuadraticTranslation(sys,WT, link_id,p_ref)
terminalCost = CostModelSum(sys, [terminalStateCost, terminalControlCost, terminalEECost])

costs = [runningCost]*T + [terminalCost]

#### Construct ILQR

In [54]:
ilqr_cost = ILQR(sys, mu)
ilqr_cost.set_init_state(x0)
ilqr_cost.set_timestep(T)
ilqr_cost.set_cost(costs)
ilqr_cost.set_state(xs, us) #set initial trajectory

In [55]:
ilqr_cost.mu = 1e-6

#### Solve and Plot

In [56]:
n_iter = 30
ilqr_cost.solve(n_iter, method='batch')
xs_batch, us_batch = ilqr_cost.xs, ilqr_cost.us

#clear_output()

1.6145078713024632
1.0 1.6335471410067526
0.8 1.628235401597161
0.6400000000000001 1.6245267641943544
0.5120000000000001 1.621906254602903
0.40960000000000013 1.6200317039673078
0.32768000000000014 1.61867413355462
0.2621440000000001 1.6176790435528552
0.2097152000000001 1.6169412111065606
0.1677721600000001 1.6163882292188205
0.13421772800000006 1.615969710997845
0.10737418240000006 1.6156501742857334
0.08589934592000005 1.6154043241792526
0.06871947673600004 1.6152139035531348
0.054975581388800036 1.6150655734795207
0.043980465111040035 1.6149494737250611
0.03518437208883203 1.61485823517401
0.028147497671065627 1.6147862947847504
0.022517998136852502 1.6147294147879234
0.018014398509482003 1.6146843410848775
0.014411518807585602 1.6146485575175624
0.011529215046068483 1.614620106920016
0.009223372036854787 1.6145974592477677
0.00737869762948383 1.6145794133065738
0.005902958103587064 1.614565022759903
0.004722366482869652 1.6145535398948263
0.0037778931862957215 1.6145443725291297
0

#### Play traj

In [33]:
sys.vis_traj(ilqr_cost.xs)

#### Compute Error

In [253]:
pos, quat = robot_sys.compute_ee(ilqr_cost.xs[-1], link_id)

In [254]:
print(pos-p_ref)

[ 0.0002  0.0015 -0.0053]
